In [7]:
%%writefile query.hql

SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=2048;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.exec.max.created.files=10000;
SET hive.error.on.empty.partition=true;

USE stackoverflow_;

DROP TABLE if exists posts_sample_external; 

CREATE EXTERNAL TABLE posts_sample_external 
(row_id string,
post_type_id string,
year string,
month string)
ROW FORMAT 
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES (
    "input.regex" = ".*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+)-(\\d+)).*"
)
LOCATION '/data/stackexchange1000/posts';

Overwriting query.hql


In [8]:
%%writefile -a query.hql

DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample 
(count int) 
PARTITIONED BY (year string, month string);

Appending to query.hql


In [9]:
%%writefile -a query.hql

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT count(*) as count, year, concat(year,"-",month) as month
WHERE year IS NOT NULL
GROUP BY year, month;

Appending to query.hql


In [10]:
%%writefile final_query.hql

USE stackoverflow_;

SELECT year, month, count 
FROM (
    SELECT ROW_NUMBER() OVER(ORDER BY year, month ASC) AS row_id, year, month, count
    FROM posts_sample
) final_table
WHERE final_table.row_id=3;

Overwriting final_query.hql


In [11]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.109 seconds
OK
Time taken: 1.57 seconds
OK
Time taken: 0.551 seconds
OK
Time taken: 3.594 seconds
OK
Time taken: 0.304 seconds
Query ID = jovyan_20180515112626_964bf1dc-3646-405f-9e86-7f09ede2411c
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526339522385_0006, Tracking URL = http://e42372799f14:8088/proxy/application_1526339522385_0006/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526339522385_0006
Hadoop job information for Stage-1: number of mappers: 1; number

Partition stackoverflow_.posts_sample{year=2011, month=2011-02} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-03} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-04} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-05} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-06} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-07} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-08} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stackoverflow_.posts_sample{year=2011, month=2011-09} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition stacko

MapReduce Jobs Launched: 
Stage-Stage-1: Map: 1  Reduce: 1   Cumulative CPU: 34.31 sec   HDFS Read: 60007573 HDFS Write: 7502 SUCCESS
Total MapReduce CPU Time Spent: 34 seconds 310 msec
OK
Time taken: 76.423 seconds


In [12]:
%%bash
hive -f final_query.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.129 seconds
Query ID = jovyan_20180515112828_f396378c-2948-459e-82fa-9ba41d36721e
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526339522385_0007, Tracking URL = http://e42372799f14:8088/proxy/application_1526339522385_0007/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526339522385_0007
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-15 11:28:25,452 Stage-1 map = 0%,  reduce = 0%
2018-05-15 11:28:34,271 Stage-1 map = 100%, 